In [ ]:
import numpy as np
import pickle
import pandas as pd
from flask import Flask, request
from flask import Flask, request, jsonify, render_template
from keras.models import load_model
import predictionlib
from base64 import b64encode
from PIL import Image
import io
import os
from gtts import gTTS
import IPython

app=Flask(__name__)
model = load_model('model_5.h5')
idx_word_dic = pickle.load(open('idx_word_dic.pkl',"rb"))
tokenizer = pickle.load(open('tokenizer.pkl', 'rb'))
max_length = pickle.load(open('max_length.pkl', 'rb'))
captions_dic =  pickle.load(open("captions_dic.pkl", 'rb'))

@app.route('/')
def home():
    return render_template('index.html')



@app.route('/predict',methods=['POST'])
def predict():
    '''
    For rendering results on HTML GUI
    '''
    f = request.files['file']
    content = f.read()
    res_image = b64encode(content).decode("utf-8")

    img = Image.open(io.BytesIO(content))
    if img.mode != 'RGB':
        img = img.convert('RGB')
    target_size = (224, 224)
    img = img.resize(target_size, Image.NEAREST)
#     print(max_length)
    caption = predictionlib.generate_captions(model, img, tokenizer, idx_word_dic, max_length)
    
    speech = gTTS(text = caption, lang = 'en', slow = False)
    print(type(speech))
#     speech.save(r'./templates/speech_1.mp3')
    
    
    IPython.display.display(IPython.display.Audio('speech_1.mp3'))

    
    return render_template('results.html', result='Predicted Caption: {}'.format(caption),
                           res_image=res_image)
    
    
    


if __name__=='__main__':
    app.run(host='0.0.0.0', port=5001)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.0.13:5001/ (Press CTRL+C to quit)
192.168.0.13 - - [28/Jan/2022 20:04:21] "GET / HTTP/1.1" 200 -
192.168.0.13 - - [28/Jan/2022 20:04:21] "GET /static/css/style.css HTTP/1.1" 304 -


<class 'gtts.tts.gTTS'>


192.168.0.13 - - [28/Jan/2022 20:04:30] "POST /predict HTTP/1.1" 200 -
192.168.0.13 - - [28/Jan/2022 20:04:31] "GET /static/css/style.css HTTP/1.1" 304 -
192.168.0.13 - - [28/Jan/2022 20:04:52] "GET /speech_1.mp3 HTTP/1.1" 404 -


In [2]:
open('speech_1.mp3', 'rb')

<_io.BufferedReader name='speech_1.mp3'>

In [1]:
!pip install pydub